In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
#    导入sklearn自带的数据集
from sklearn.datasets import load_iris
#    导入二叉树
#from binarytree import Node

In [2]:
iris = load_iris()
X = iris.data
y = iris.target
#    分割为训练集、测试集
X_train,X_test, y_train, y_test = train_test_split(X,y,test_size=0.4, random_state=5)
#    数据标准化
X_train = (X_train - X_train.mean())/X_train.std()
X_test = (X_test - X_test.mean())/X_test.std()

构造平衡kd树

In [3]:
#    kd树
kd_tree = []
k = X.shape[1]

#    树节点
class Node:
    '''
    定义树节点结构
    '''
    def __init__(self, value=None, left=None, right=None, parent=None, axis=None, label=None):
        self.value = value
        self.left = left
        self.right = right
        self.parent = parent
        self.axis = axis
        self.label = label


def and_kd_tree(node, depth):
    '''
    构造kd树
    node: 当前节点
    depth: 当前节点深度
    parent：当前节点的父节点
    '''
    #print(node.value)
    if node == None:
        return None
    #    选择要划分的轴
    l = depth%k 
    #    第l列的中位数
    data_cs = node.value[node.value[:, l].argsort()]
    median = data_cs[data_cs.shape[0]>>1, l]
    #    左子节点
    node_left = []
    #    右子节点
    node_right = []
    cnt = 1
    node_value = None
    for i in range(node.value.shape[0]):
        if node.value[i,l] == median and cnt:
            node_value = node.value[i]       
            cnt = 0
        #    若大于中位数，则扩充至右子节点
        elif node.value[i,l] >= median:
            node_right.append(node.value[i])
        #    若小于中位数，则扩充至左子节点
        else:
            node_left.append(node.value[i])       
    
    #    转为数组形式
    node_right = np.array(node_right)
    node_left = np.array(node_left)
    #    构造节点
    node.value = node_value[0:-1]
    
    node.axis = l
    node.label = node_value[-1]
    if len(node_right) != 0:
        node.right = Node(node_right, parent=node)
    #else:
    #    node.right = None
    if len(node_left) != 0:
        node.left = Node(node_left, parent=node)
    #else:
    #    node.left = None
    #    递归
    and_kd_tree(node.right, depth+1)
    and_kd_tree(node.left, depth+1)
    
#    前序遍历测试   
def pres(node):
    
    if node is None:
        return
    print(node.parent)
    pres(node.left)
    pres(node.right)
    


In [4]:
#    将feature和label合并 构造树节点
data_train = np.column_stack((X_train,y_train))

#    根节点
root = Node(data_train)
#    深度
depth = 0
#    构造kd树
and_kd_tree(root,depth)

搜索kd树

In [5]:
#    深度
depth = 0
def curr_near_node(x, node):
    '''
    递归求当前最邻近点（叶节点），实现算法3.3中的(1),(2)
    x: 目标点
    depth：当前节点的深度
    node: 当前节点
    '''
    near_node = node
    #l = depth%k 
    #depth += 1
    #print(node.value)
    #    划分的轴
    l = node.axis
    if x[l] >= node.value[l]:
        if node.right is not None:
            near_node = curr_near_node(x, node.right)
        elif node.left is not None:
            near_node = curr_near_node(x, node.left)
        else:
            near_node = node
    else:
        if node.left is not None:
            near_node = curr_near_node(x, node.left)
        elif node.right is not None:
            near_node = curr_near_node(x, node.right)
        else:
            near_node = node
    return near_node

#    测试
x = X_test[0]
print(x)
node = curr_near_node(x, root)
node.value

[ 1.20308351 -0.38424426  0.2302052  -1.15230609]


array([ 1.17032169, -0.38936186,  0.31501136, -1.2446722 ])

In [6]:
def euc_dist(xi,xj):
    '''
    距离度量（Euclidean distance）：欧氏距离
    '''
    dist = 0
    for i in range(len(xi)):
        dist += (xi[i] - xj[i])**2
    return dist**0.5

In [7]:
print(euc_dist(x,node.value))

0.12970392786999457


In [8]:
def call_back_root(x, node_xn, node, access_node, root):
    '''
    递归回退到根节点,实现算法3.3中的(3)
    x: 目标节点的值
    node_xn: 当前最近节点
    node: 当前节点
    access_node: 已访问过的节点
    root: 根节点，回退结束的标志
    '''
    if node.parent is None:
        #print(xn)
        return node_xn
    #    最近节点
    #    实现(3)(a)
    if euc_dist(x, node_xn.value) >= euc_dist(x, node.value):
        
        node_xn = node
    #    表示已访问过
    access_node.append(node)
    #    划分的轴
    l = node.axis
    another_node = None
    if node.parent.left == node:
        #    则另一个结点是右节点
        another_node = node.parent.right
    else:
        #    则另一个节点是左节点
        another_node = node.parent.left
    if another_node is not None and another_node not in access_node:
        if abs(another_node.value[l] - x[l]) > euc_dist(x, node_xn.value):
            #    则相交，表明该节点区域有可能存在最邻近节点，
            #    递归进行最近邻搜索
            n_node = curr_near_node(x, another_node)
            node_xn = call_back_root(x, node_xn, n_node, access_node, root)
    #print(another_node)
    node_xn = call_back_root(x, node_xn, node.parent, access_node, root)
    
    return node_xn

判断超球体与分类矩形超平面是否相交的方法为：待分类点（目标点）与分类超矩形平面的距离（即在当前分类变量上，待分类点与父结点在上取值的差值的绝对值）是否大于超球体半径

In [9]:
access_node = []
near_xn = None
near_nx = call_back_root(x, node, node, access_node, root)

near_nx.value, euc_dist(near_nx.value, x)

(array([ 1.17032169, -0.38936186,  0.31501136, -1.2446722 ]),
 0.12970392786999457)

In [10]:

predict = []
for i in range(len(X_test)):
    
    x = X_test[i]
    #    当前最近点 叶节点
    node = curr_near_node(x, root)
    
    access_node = []
    near_xn = None
    #    回退到根节点 并求得最近邻
    near_nx = call_back_root(x, node, node, access_node, root)
    predict.append(near_nx.label)
    #print(near_nx.label)

correct = [1 if i==j else 0 for (i,j) in zip(predict, list(y_test))]
correct.count(1)/len(correct)

0.9666666666666667